In [2]:
from models.yolov3_gpu_head.inference import restore_model


restored_model = restore_model()

/Users/martingalajda/School/DIPLOMA-THESIS/object-detection/venv/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
import keras.backend as K

yolo_head_1_shape = K.shape(restored_model.outputs[0])
yolo_head_2_shape = K.shape(restored_model.outputs[1])
yolo_head_3_shape = K.shape(restored_model.outputs[2])

def get_yolo_cols_and_rows(yolo_out_shape):
    return yolo_out_shape[1], yolo_out_shape[2]

yolo_head_1_grid_cols, yolo_head_1_grid_cols = get_yolo_cols_and_rows(yolo_head_1_shape)
yolo_head_2_grid_cols, yolo_head_2_grid_cols = get_yolo_cols_and_rows(yolo_head_2_shape)
yolo_head_3_grid_cols, yolo_head_3_grid_cols = get_yolo_cols_and_rows(yolo_head_3_shape)

K.shape(restored_model.outputs)

<tf.Tensor 'Shape_3:0' shape=(5,) dtype=int32>

In [ ]:
import keras.backend as K

fake_img_tensor = K.arange(0, 255)

fake_img_tensor = K.concatenate([fake_img_tensor, fake_img_tensor, fake_img_tensor], axis=0)
fake_img_tensor = K.reshape(fake_img_tensor, [1, 255, 3])

# fake_img_tesnor = K.backend.reshape(fake_img_tensor, [1, ])
K.eval(K.shape(fake_img_tensor))


In [ ]:
import keras.backend as K

num_of_cols, num_of_rows = 19, 15
test_tensor = K.arange(0, num_of_cols)

test_tensor = K.reshape(test_tensor, [1, num_of_cols, 1, 1])
test_tensor = K.tile(test_tensor, [1, num_of_rows, 1, 1])
test_tensor = K.reshape(test_tensor, [1, num_of_rows, num_of_cols, 1])

K.eval(K.shape(test_tensor / K.shape(test_tensor)[::-1]))
K.eval(K.shape(test_tensor)[::])
K.eval(test_tensor / K.shape(test_tensor)[::-1])
K.eval(test_tensor / K)

In [4]:
from utils.image import load_pil_image_from_file
from utils.preprocess_image import resize_and_letter_box
import numpy as np

PATH_TO_TEST_FILE = '../resources/horse.jpeg'

orig_img_pil, img_np = load_pil_image_from_file(PATH_TO_TEST_FILE)
print(f'{orig_img_pil.width}, {orig_img_pil.height}')
img_np = resize_and_letter_box(img_np/255., target_width = 608, target_height = 608)
img_np = np.expand_dims(img_np, 0)


284, 177


In [10]:
def get_corrected_boxes(*, box_width, box_height, box_x, box_y, orig_image_shape, model_image_shape):
    orig_image_w, orig_image_h = orig_image_shape
    model_w, model_h = model_image_shape

    if float(model_w / orig_image_w) < float(model_h / orig_image_h):
        new_w = model_w
        new_h = (orig_image_h * model_w) / orig_image_w
    else:
        new_h = model_h
        new_w = (orig_image_w * model_h) / orig_image_h

    box_x = (box_x - (((model_w - new_w)/2.0)/model_w)) / float(new_w/model_w)
    box_y = (box_y - (((model_h - new_h)/2.0)/model_h)) / float(new_h/model_h)

    box_width *= model_w/new_w
    box_height *= model_h/new_h

    left = (box_x - (box_width/2.)) * orig_image_w
    right = (box_x + (box_width/2.)) * orig_image_w
    top = (box_y - (box_height/2.)) * orig_image_h
    bottom = (box_y + (box_height/2.)) * orig_image_h

    output_box = [
        max(int(left), 0),
        max(int(top), 0),
        min(int(right), orig_image_w),
        min(int(bottom), orig_image_h)
    ]

    return output_box


def keras_get_corrected_boxes(*, box_width, box_height, box_x, box_y, orig_image_shape, model_image_shape):
    orig_image_w, orig_image_h = orig_image_shape[0], orig_image_shape[1]
    model_w, model_h = model_image_shape[0], model_image_shape[1]

    scale = K.min(K.concatenate([(model_w/orig_image_w), (model_h/orig_image_h)]))
    w_without_padding = orig_image_w * scale
    h_without_padding = orig_image_h * scale

#     if float(model_w / orig_image_w) < float(model_h / orig_image_h):
#         w_without_padding = model_w
#         h_without_padding = (orig_image_h) * model_w / orig_image_w
#     else:
#         h_without_padding = model_h
#         w_without_padding = (orig_image_w) * model_h / orig_image_h
        
    x_shift = (model_w - w_without_padding)/2.0/ model_w
    y_shift = (model_h - h_without_padding)/2.0/ model_h

    box_x = (box_x - x_shift) / (w_without_padding / model_w)
    box_y = (box_y - y_shift) / (h_without_padding / model_h)

    box_width *= model_w/w_without_padding
    box_height *= model_h/h_without_padding

    left = (box_x - (box_width/2.))  * orig_image_w
    right = (box_x + (box_width/2.))  * orig_image_w
    top = (box_y - (box_height/2.))  * orig_image_h
    bottom = (box_y + (box_height/2.))  * orig_image_h

    output_boxes = K.concatenate([
        K.reshape(left, [-1, 1]),
        K.reshape(top, [-1, 1]),
        K.reshape(right, [-1, 1]),
        K.reshape(bottom, [-1, 1])
    ])

    return output_boxes



In [12]:
import numpy as np
import keras.backend as K
from utils.non_max_suppression import non_max_suppression_fast, classic_non_max_suppression
from models.utils.visualize import draw_detected_boxes_on_pil_image_v2
import time
import tensorflow as tf


start = time.time()
sess = K.get_session()

NUM_OF_CLASSES = 601
NUM_OF_ANCHORS = 3
NUM_OF_BOX_PARAMS = 5
ANCHORS = np.array([
    [[116, 90], [156, 198], [373, 326]],
    [[30, 61], [62, 45], [59, 119]],
    [[10, 13], [16, 30], [33, 23]]
]) / np.array([608., 608.])

reshaped_heads = []

orig_image_width, orig_image_height =  orig_img_pil.width, orig_img_pil.height

prob_obj = []
prob_class = []
prob_detected = []
boxes = []

def get_grid(rows, cols):
    grid_x = K.arange(0, stop=cols)
    grid_x = K.reshape(grid_x, [1, -1, 1, 1])
    grid_x = K.tile(grid_x, [rows, 1, 1, 1])
    
    grid_y = K.arange(0, stop=rows)
    grid_y = K.reshape(grid_y, [-1, 1, 1, 1])
    grid_y = K.tile(grid_y, [1, cols, 1, 1])
    
    grid = K.concatenate([grid_x, grid_y])
    
    return grid

placeholder_orig_image_width = K.placeholder(shape=(1,))
placeholder_orig_image_height = K.placeholder(shape=(1,))
 
for yolo_head_idx in range(len(restored_model.output)):
    yolo_head = restored_model.output[yolo_head_idx]
    yolo_head_shape = K.shape(yolo_head)
    yolo_head_num_of_cols, yolo_head_num_of_rows = yolo_head_shape[2], yolo_head_shape[1]
    
    curr_yolo_head = K.reshape(yolo_head, [-1, yolo_head_num_of_cols, yolo_head_num_of_rows, NUM_OF_ANCHORS, NUM_OF_BOX_PARAMS+NUM_OF_CLASSES])
    grid = K.cast(get_grid(yolo_head_shape[1], yolo_head_shape[2]), dtype=K.dtype(curr_yolo_head))
    
    curr_boxes_xy = (K.sigmoid(curr_yolo_head[..., :2]) + grid) / K.cast([yolo_head_num_of_cols, yolo_head_num_of_rows],  dtype=K.dtype(curr_yolo_head))
    curr_boxes_wh = K.exp(curr_yolo_head[..., 2:4]) * ANCHORS[yolo_head_idx]
    
    curr_prob_obj = K.sigmoid(curr_yolo_head[..., 4:5])
    curr_prob_class = K.sigmoid(curr_yolo_head[..., 5:])
    
    curr_prob_detected_class = curr_prob_obj * curr_prob_class

    boxes.append(keras_get_corrected_boxes(
        box_width=curr_boxes_wh[..., 0:1],
        box_height=curr_boxes_wh[..., 1:2],
        box_x=curr_boxes_xy[..., 0:1],
        box_y=curr_boxes_xy[..., 1:2],
        orig_image_shape=(placeholder_orig_image_width, placeholder_orig_image_height),
        model_image_shape=(608, 608)
    ))
    
    curr_prob_detected_class = K.reshape(curr_prob_detected_class, [-1, NUM_OF_CLASSES])
    prob_class.append(curr_prob_detected_class)

    
prob_class = K.concatenate(prob_class, axis = 0)
boxes = K.concatenate(boxes, axis=0)

mask = prob_class >= 0.25
max_boxes_tensor = K.constant(20, dtype='int32')

boxes_ = []
scores_ = []
classes_ = []

for c in range(NUM_OF_CLASSES):
    # TODO: use keras backend instead of tf.
    class_boxes = tf.boolean_mask(boxes, mask[:, c])
    class_box_scores = tf.boolean_mask(prob_class[:, c], mask[:, c])
    nms_index = tf.image.non_max_suppression(class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=0.5)
    
    
    class_boxes = K.gather(class_boxes, nms_index)
    class_box_scores = K.gather(class_box_scores, nms_index)
    classes = K.ones_like(class_box_scores, 'int32') * c
    
    boxes_.append(class_boxes)
    scores_.append(class_box_scores)
    classes_.append(classes)
    
boxes_ = K.concatenate(boxes_, axis=0)
scores_ = K.concatenate(scores_, axis=0)
classes_ = K.concatenate(classes_, axis=0)


# out_tensors = [
#     boxes,
#     prob_class,
# ]

out_tensors = [
    boxes_,
    scores_,
    classes_
]


print(f'Took {time.time() - start} seconds to construct network.')

start = time.time()
sess_out = sess.run(out_tensors, feed_dict={
    restored_model.input: img_np,
    placeholder_orig_image_width: [orig_image_width],
    placeholder_orig_image_height: [orig_image_height],
    K.learning_phase(): 0
})
boxes, scores, out_boxes_classes = sess_out

    
all_curr_detected_objects = boxes
all_curr_detected_classes = scores
all_curr_detected_scores = out_boxes_classes

# for c in range(NUM_OF_CLASSES):
#     curr_mask_detected = out_boxes_classes[..., c] > 0.5
#     curr_probs_class = out_boxes_classes[curr_mask_detected, :][:, c]
#     c_boxes = boxes[curr_mask_detected, :]
    
#     curr_detected_objects = []
#     curr_detected_classes = []
#     curr_detected_scores = []

#     for idx in range(np.count_nonzero(curr_mask_detected)):
#         box_class_prob = curr_probs_class[idx]

#         curr_detected_objects += [c_boxes[idx]]
#         curr_detected_classes += [c]
#         curr_detected_scores += [box_class_prob]
        
#     if len(curr_detected_objects) > 0:
#         print(f'np.array(curr_detected_objects) = {np.array(curr_detected_objects)}')

#         chosen_box_indices = classic_non_max_suppression(curr_detected_objects, curr_detected_scores, 0.6)
#         curr_detected_objects = [curr_detected_objects[i] for i in chosen_box_indices]
#         curr_detected_classes = [curr_detected_classes[i] for i in chosen_box_indices]
#         curr_detected_scores = [curr_detected_scores[i] for i in chosen_box_indices]
        
#         all_curr_detected_objects += curr_detected_objects
#         all_curr_detected_classes += curr_detected_classes
#         all_curr_detected_scores += curr_detected_scores


#     print(np.array(out).shape)
    
print(f'Took {time.time() - start} seconds')
print(f'np.array(all_curr_detected_objects) = {np.array(all_curr_detected_objects)}')
print(f'np.array(all_curr_detected_classes) = {np.array(all_curr_detected_classes)}')
print(f'np.array(all_curr_detected_scores) = {np.array(all_curr_detected_scores)}')

draw_detected_boxes_on_pil_image_v2(all_curr_detected_objects, all_curr_detected_classes, np.array(all_curr_detected_scores), orig_img_pil)
orig_img_pil.show()

Took 30.346962213516235 seconds to construct network.
Took 27.88591504096985 seconds
np.array(all_curr_detected_objects) = [[ 34.637608  10.438645 279.33963  174.30429 ]]
np.array(all_curr_detected_classes) = [0.9730875]
np.array(all_curr_detected_scores) = [560]
0.9730875


In [6]:
K.backend.eval()

AttributeError: 'function' object has no attribute 'eval'

In [12]:
ANCHORS[0]

array([[0.19078947, 0.14802632],
       [0.25657895, 0.32565789],
       [0.61348684, 0.53618421]])

In [111]:
K.eval(K.concatenate([
    K.constant([[1], [1]]),
    K.constant([[2], [2]])
], axis = 0))


array([[1.],
       [1.],
       [2.],
       [2.]], dtype=float32)